# 🚀 Production Deployment: VeganFlow to Vertex AI Agent Engine

This notebook automates the deployment of the **VeganFlow: Autonomous Supply Chain Intelligence** system to Google Cloud.

**What we are doing:**
1.  **Configuring** the cloud resources (CPU/Memory).
2.  **Creating an Entry Point** (`agent.py`) that bridges our package structure to the Agent Engine runtime.
3.  **Deploying** the entire `veganflow_ai` ecosystem using the ADK CLI.
4.  **Verifying** the deployment by running a live cloud query.

**Architecture:**
We are deploying a **Hierarchical Multi-Agent System** where the state (SQLite) is re-initialized on startup to ensure robustness in a stateless cloud environment.
* **Deployment Target:** Vertex AI Agent Engine.
* **Source:** The `veganflow_ai` package folder.
* **Hardware:** 1 CPU, 2GB RAM (Required to support the Multi-Agent orchestration and SQLite DB).

**Prerequisites:**
* Google Cloud Project with Billing enabled.
* APIs enabled: Vertex AI, Cloud Build, Cloud Run, Artifact Registry.

##### Uncomment and run below to set .env file

###### 🛑 ACTION REQUIRED (IF you have not set the .env file)
1.  Uncomment and run the below cell
2.  Open the `veganflow_ai/.env` file generated below.
3.  **Replace** the placeholders with your actual API Key and Project ID.
4.  **Save** the file.

In [2]:
# %%writefile veganflow_ai/.env
# # ==============================================
# # 🔑 VEGANFLOW CONFIGURATION
# # ==============================================

# # 1. Set to 1 to use Vertex AI, or 0 to use Google AI Studio (Set GOOGLE_API_KEY accordingly)
# GOOGLE_GENAI_USE_VERTEXAI=1
# #GOOGLE_API_KEY="PASTE_YOUR_GEMINI_API_KEY_HERE"

# # 2. Google Cloud Config (Required for Vertex AI Memory Bank & Tracing)
# # Get this from your Google Cloud Console
# GOOGLE_CLOUD_PROJECT="PASTE_YOUR_PROJECT_ID_HERE"
# GOOGLE_CLOUD_LOCATION="us-central1"

# # 3. (Optional) Telemetry
# # Set to 'true' if you want to see detailed traces in the console/Cloud Trace
# ENABLE_TRACING=true

# # 4. AGENT ENGINE ID (Added after first successful deployment)
# # You will get this ID *after* you run the deployment notebook.
# # Required for: Connecting to the persistent cloud memory service.
# AGENT_ENGINE_ID=""

In [3]:
import os
import sys
from dotenv import load_dotenv

# 1. Load Environment Variables
env_path = "veganflow_ai/.env"
load_dotenv(env_path, override=True)

# 2. Configuration
# We fetch the Project ID defined in your .env file
PROJECT_ID = os.getenv("GOOGLE_CLOUD_PROJECT")
# Default to us-central1 if not set, or fetch from env
REGION = os.getenv("GOOGLE_CLOUD_LOCATION", "us-central1")

if not PROJECT_ID:
    raise ValueError("❌ Error: GOOGLE_CLOUD_PROJECT not found in .env file.")

print(f"✅ Target Project: {PROJECT_ID}")
print(f"✅ Target Region: {REGION}")

# 3. Authenticate
# This ensures the notebook has permission to deploy
print("\n🔐 Checking Google Cloud Credentials...")
!gcloud auth application-default login

✅ Target Project: gen-lang-client-0353405573
✅ Target Region: us-central1

🔐 Checking Google Cloud Credentials...
Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=764086051850-6qr4p6gpi6hn506pt8ejuq83di341hur.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8085%2F&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login&state=cG7Mk1R2jn40H5xGUP0qKW2Ck5ndzT&access_type=offline&code_challenge=61sVwHlfc_ZMPUzWoxEfp7VOZUtKq7EgSlVBecFlDbY&code_challenge_method=S256


Credentials saved to file: [/Users/karthicksothivelr/.config/gcloud/application_default_credentials.json]

These credentials will be used by any library that requests Application Default Credentials (ADC).

Quota project "gen-lang-client-0353405573" was added to ADC which can be used by Google client libraries for billi

## 🛠️ Step 1: Define Hardware Resources
We create the configuration file **inside the package folder** (`veganflow_ai/`) so it gets uploaded with the code. We allocate 2GB RAM to handle the multi-agent overhead.

In [5]:
import json

config = {
    "min_instances": 0,
    "max_instances": 1,
    "resource_limits": {
        "cpu": "1",
        "memory": "2Gi"
    }
}

# Write to the package directory
with open("veganflow_ai/.agent_engine_config.json", "w") as f:
    json.dump(config, f, indent=4)

print("✅ Configuration file created at: veganflow_ai/.agent_engine_config.json")

✅ Configuration file created at: veganflow_ai/.agent_engine_config.json


## 🔌 Step 2: Create the Cloud Entry Point (`agent.py`)
We create the `agent.py` file **inside** `veganflow_ai/`.

**Critical Note:** We use **absolute imports** (`from agents...`) instead of relative imports (`from .agents...`) because this file becomes the root script in the cloud container.

In [6]:
%%writefile veganflow_ai/agent.py
import logging
import sys
import os

# --- Add current directory to Python Path ---
# This ensures that 'agents' and 'tools' can be imported as top-level modules
# whether running locally via 'adk web' or deployed in the cloud.
current_dir = os.path.dirname(os.path.abspath(__file__))
if current_dir not in sys.path:
    sys.path.append(current_dir)

# --- 1. Import Modules ---
from agents.orchestrator import create_store_manager
from tools.retail_database_setup import setup_retail_database

# --- 2. Cloud Initialization ---
# Re-create the ephemeral database every time the agent starts
print("📦 [Cloud Init] Setting up Retail Database schema...")
setup_retail_database()

# --- 3. Define Root Agent ---
# Agent Engine looks for the variable 'root_agent'
print("🤖 [Cloud Init] Initializing VeganFlow Orchestrator...")
root_agent = create_store_manager()

print("✅ [Cloud Init] System Ready.")

Overwriting veganflow_ai/agent.py


## 📦 Step 3: Manage Dependencies
We ensure the deployment package has its own `requirements.txt`.

In [7]:
# Copy the root requirements to the package folder
!cp requirements.txt veganflow_ai/requirements.txt
print("✅ Copied requirements.txt to deployment folder.")

✅ Copied requirements.txt to deployment folder.


## 🚀 Step 4: Deploy!
We run the `adk deploy` command pointing specifically to the `veganflow_ai` directory.

In [8]:
# Deploy the package folder
# This may take 3-5 minutes
!adk deploy agent_engine \
  --project="$PROJECT_ID" \
  --region="$REGION" \
  --agent_engine_config_file=veganflow_ai/.agent_engine_config.json \
  veganflow_ai

Staging all files in: /Users/karthicksothivelr/Downloads/Autonomous_Supply_Chain_Intelligence/veganflow_ai_tmp20251130_093439
Copying agent source code...
Copying agent source code complete.
Resolving files and dependencies...
Reading agent engine config from veganflow_ai/.agent_engine_config.json
Reading environment variables from /Users/karthicksothivelr/Downloads/Autonomous_Supply_Chain_Intelligence/veganflow_ai/.env
Ignoring GOOGLE_CLOUD_PROJECT in .env as `--project` was explicitly passed and takes precedence
Ignoring GOOGLE_CLOUD_LOCATION in .env as `--region` was explicitly passed and takes precedence
Initializing Vertex AI...
Vertex AI initialized.
Created veganflow_ai_tmp20251130_093439/agent_engine_app.py
Files and dependencies resolved
Deploying to agent engine...
✅ Created agent engine: projects/282554295020/locations/us-central1/reasoningEngines/2842804905800892416
Cleaning up the temp folder: veganflow_ai_tmp20251130_093439


[Agent Engine Console](https://console.cloud.google.com/vertex-ai/agents/agent-engines)

## 🧪 Step 5: Post-Deployment Verification
1.  **Copy** the **Agent Engine ID** from the output above (it looks like `projects/.../reasoningEngines/123456...`) --> Just copy the AGENT ENGINE ID (after reasoningEngines/).
2.  **Paste** it below to run a live smoke test against the cloud agent.

In [19]:
import vertexai
from vertexai import agent_engines

# ---------------------------------------------------------
# 🚨 ACTION REQUIRED: Paste your Agent Engine ID here
# ---------------------------------------------------------
AGENT_ENGINE_ID = "REPLACE_WITH_YOUR_ID_FROM_OUTPUT_ABOVE"
AGENT_ENGINE_ID = "2842804905800892416" 

if "REPLACE" in AGENT_ENGINE_ID:
    print("⚠️ WAITING: Please paste the Agent Engine ID from the deployment output above.")
else:
    # Initialize Vertex AI (Uses Application Default Credentials)
    vertexai.init(project=PROJECT_ID, location=REGION)
    
    # Construct full resource name
    resource_name = f"projects/{PROJECT_ID}/locations/{REGION}/reasoningEngines/{AGENT_ENGINE_ID}"
    
    try:
        # Connect to the cloud agent
        remote_agent = agent_engines.get(resource_name)
        print("🆔 Creating Cloud Session...")
        USER_ID = "ops_manager"
        remote_session = remote_agent.create_session(user_id=USER_ID)
        SERVER_SESSION_ID = remote_session["id"]

        # Run a live test query
        query = "Check the stock level for Oat Barista Blend."
        print(f"📤 Sending Cloud Query: '{query}'")
        print("-" * 40)
        
        # Stream the response
        async for event in remote_agent.async_stream_query(message=query, user_id=USER_ID, session_id=SERVER_SESSION_ID):
            print(event)
            
        print("-" * 40)
        print("✅ Cloud Smoke Test Complete")
        
    except Exception as e:
        print(f"\n❌ Error connecting to agent: {e}")

🆔 Creating Cloud Session...
📤 Sending Cloud Query: 'Check the stock level for Oat Barista Blend.'
----------------------------------------
{'model_version': 'gemini-2.5-pro', 'content': {'parts': [{'function_call': {'id': 'adk-584d7416-3eb6-4339-b933-b428a5a9626b', 'args': {'agent_name': 'shelf_monitor'}, 'name': 'transfer_to_agent'}, 'thought_signature': 'CpgDAePx_14GVJ3aUlWJprWZ5IQp5mncLfM2GzVVqNqZoISUVLUZ3Br_10q2DMgmfs4Yw1Op9_Qr-JUjxZZClf9DojB2nJLJNtdbkWkpZvsr525AoIbTMygRegPbXS7fOAqZjEPefYDbRtuivm1YZNTAAufWCtiqMhUSBKBWiwCilVR__uKoxFrhyPvcvKsjb9zw6U_8s7LQDUx5cuoiv_QgJ73iqev-X9K4qJFLP5TLXVxsOCiI-OdrwQyB2D1g3rjYkD5l4jWyyN50H8hoaKuk5H-6N1jTcjhV_RIZ5aZ0vx_wz0mG9XIeh7n8p_Nh-hh9zuCZxkTnjJaGt6ltCFUoNnRPMrg1Cue0HKXNJd6SVegndz14KqMWnKZfXWoP0Nk35ZfB5fgWH3cretIJT8sdOrlKt2wAdlARLF62qo7g_9OZ-nFGbJK8mxbRnxEkrornx372UdrolDdc0xZECH5ppOhyZCZ2scjbPbC4QRnYdJtCSBx-19Jj83uAYIYwaBg-0-UblJA93TjHqtnvZddJkg4fHDz5jd7Y'}], 'role': 'model'}, 'finish_reason': 'STOP', 'usage_metadata': {'candidates_token_count': 

---

## 🧹 Section 6: Cleanup

**⚠️ IMPORTANT: Prevent unexpected charges: Always delete resources when done testing!**

**Cost Reminders**

As a reminder, leaving the agent running can incur costs. Agent Engine offers a monthly free tier, which you can learn more about in the [documentation](https://docs.cloud.google.com/agent-builder/agent-engine/overview#pricing).

**Always delete resources when done testing!**

When you're done testing and querying your deployed agent, it's recommended to delete your remote agent to avoid incurring additional costs:

In [7]:
agent_engines.delete(resource_name=remote_agent.resource_name, force=True)

print("✅ Agent successfully deleted")

Deleting AgentEngine resource: projects/gen-lang-client-0353405573/locations/us-central1/reasoningEngines/3552121847111745536
Delete AgentEngine backing LRO: projects/282554295020/locations/us-central1/operations/8420608478472568832
AgentEngine resource deleted: projects/gen-lang-client-0353405573/locations/us-central1/reasoningEngines/3552121847111745536
✅ Agent successfully deleted
